In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math

# for circuit construction
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# QFT circuit needed for phase estimation
from qiskit.circuit.library import QFT

# for creating custom gates
from qiskit import quantum_info as qi

# for simulation
from qiskit_aer import Aer
from qiskit import transpile
from qiskit.visualization import plot_histogram

# for storing data later
import pandas as pd

In [2]:
%run 'hamiltonian.ipynb' ## to get the hamiltonian cycles function

cities = ["A", "B", "C", "D", "E"]
cycles = hamiltonian_cycles(cities, symmetric = True)

In [3]:
cycles

['ABCDEA',
 'ABCEDA',
 'ABDCEA',
 'ABDECA',
 'ABECDA',
 'ABEDCA',
 'ACBDEA',
 'ACBEDA',
 'ACDBEA',
 'ACEBDA',
 'ADBCEA',
 'ADCBEA']

In [4]:
def hamiltonian_sum(cycle, edge_weights):
    return sum(edge_weights[(list(cycle)[i], list(cycle)[i+1])] for i in range(len(cycle) - 1))

In [5]:
edge_weights = {
    ('A', 'B'): 10,
    ('A', 'C'): 5,
    ('A', 'D'): 7,
    ('A', 'E'): 9,
    ('B', 'C'): 1,
    ('B', 'D'): 5,
    ('B', 'E'): 4,
    ('C', 'D'): 8,
    ('C', 'E'): 1,
    ('D', 'E'): 3,
}

# ensuring edge-weights are bi-directional
for (city1, city2), weight in list(edge_weights.items()):
    edge_weights[(city2, city1)] = weight

cycle_weights = [hamiltonian_sum(cycle, edge_weights) for cycle in cycles]
cycle_weights

[31, 22, 33, 24, 30, 30, 23, 20, 31, 22, 23, 29]

In [6]:
## weights not satisfying constraints set to zero

edge_weights_c = {
    ('A', 'B'): 0,
    ('A', 'C'): 5,
    ('A', 'D'): 7,
    ('A', 'E'): 0,
    ('B', 'C'): 1,
    ('B', 'D'): 5,
    ('B', 'E'): 4,
    ('C', 'D'): 8,
    ('C', 'E'): 1,
    ('D', 'E'): 3,
}

for (city1, city2), weight in list(edge_weights_c.items()):
    edge_weights_c[(city2, city1)] = weight
    
cycle_weights_c = [hamiltonian_sum(cycle, edge_weights_c) for cycle in cycles]
cycle_weights_c

[12, 12, 14, 14, 20, 20, 14, 20, 22, 22, 14, 20]

In [7]:
#solutions
indices = np.where(np.array(cycle_weights) == np.array(cycle_weights_c))[0]
print(indices)
np.array(cycles)[indices]

[7 9]


array(['ACBEDA', 'ACEBDA'], dtype='<U6')

In [8]:
weights = list(edge_weights.values())

#sorting edge weights
sorted_weights = np.sort(weights)[::-1]

# normalization factor
S = np.sum(sorted_weights[:4])

# epsilon
eps = 2

weights = weights / (S + eps)
weights

array([0.25 , 0.125, 0.175, 0.225, 0.025, 0.125, 0.1  , 0.2  , 0.025,
       0.075, 0.25 , 0.125, 0.175, 0.225, 0.025, 0.125, 0.1  , 0.2  ,
       0.025, 0.075])

In [9]:
for (city1, city2), weight in list(edge_weights.items()):
    edge_weights[(city1, city2)] = weight/ (S + eps)

for (city1, city2), weight in list(edge_weights_c.items()):
    edge_weights_c[(city1, city2)] = weight/ (S + eps)

cycle_weights = [hamiltonian_sum(cycle, edge_weights) for cycle in cycles]
cycle_weights_c = [hamiltonian_sum(cycle, edge_weights_c) for cycle in cycles]

sols = np.zeros((12,2))
for i, (cycle, cycle_c) in enumerate(zip(cycle_weights, cycle_weights_c)):
    sols[i][0] = np.round(cycle,4)
    sols[i][1] = np.round(cycle_c,4)

sols

array([[0.775, 0.3  ],
       [0.55 , 0.3  ],
       [0.825, 0.35 ],
       [0.6  , 0.35 ],
       [0.75 , 0.5  ],
       [0.75 , 0.5  ],
       [0.575, 0.35 ],
       [0.5  , 0.5  ],
       [0.775, 0.55 ],
       [0.55 , 0.55 ],
       [0.575, 0.35 ],
       [0.725, 0.5  ]])

In [10]:
A,  B,   C,  D,  E = [], [], [], [], []
Ac, Bc, Cc, Dc, Ec = [], [], [], [], []

# Lists to hold all the lists
U  = [A, B, C, D, E] # before constraint
Uc = [Ac, Bc, Cc, Dc, Ec] # after constraint

for i, (city,city_c) in enumerate(zip(U,Uc)):
    for j in range(5):
        if i != j:
            city.append(np.exp(1j * edge_weights[(cities[i], cities[j])] * 2 * np.pi))
            city_c.append(np.exp(1j * edge_weights_c[(cities[i], cities[j])] * 2 * np.pi))
        else:
            city.append(1)
            city_c.append(1)
            
U  = np.kron(np.kron(np.kron(np.kron(A,B),C),D),E)
Uc = np.kron(np.kron(np.kron(np.kron(Ac,Bc),Cc),Dc),Ec)


m = 12 # number of eigenstate qubits needed
filler = 2**12 - U.shape[0] #filler ones needed to have exactly 12 qubits

U = np.append(U, np.ones(filler))
Uc = np.append(Uc, np.ones(filler))

In [11]:
table = sort_indices(*map_indices(cities, symmetric = True))

# Base 10 and 2 conversions
table = np.array(table)
indices = table[:,2]
base_10 = np.array([int(indices[i], len(cities)) for i in range(len(indices))])
base_2  = np.array(["{0:b}".format(base_10[i]) for i in range(len(base_10))]) 
table = np.append(table, base_10.reshape(-1,1), axis=1)
table = np.append(table, base_2.reshape(-1,1), axis=1)
print("cycle, index, sorted_index, base 10, base 2 \n",table)

cycle, index, sorted_index, base 10, base 2 
 [['ABCDEA' '12340' '12340' '970' '1111001010']
 ['ABCEDA' '12430' '12403' '978' '1111010010']
 ['ABDCEA' '13240' '13420' '1110' '10001010110']
 ['ABDECA' '13420' '13042' '1022' '1111111110']
 ['ABECDA' '14230' '14302' '1202' '10010110010']
 ['ABEDCA' '14320' '14023' '1138' '10001110010']
 ['ACBDEA' '21340' '23140' '1670' '11010000110']
 ['ACBEDA' '21430' '24103' '1778' '11011110010']
 ['ACDBEA' '23140' '24310' '1830' '11100100110']
 ['ACEBDA' '24130' '23401' '1726' '11010111110']
 ['ADBCEA' '31240' '32410' '2230' '100010110110']
 ['ADCBEA' '32140' '34120' '2410' '100101101010']]


In [12]:
## confirming where the solutions are in matrix-U and constrained matrix-Up

table = np.array(table)
indices = table[:,2]
base_10 = np.array([int(indices[i], len(cities)) for i in range(len(indices))])
checks = np.angle(U[base_10])/2/np.pi + 1
checks_c = np.angle(Uc[base_10])/2/np.pi
checks[7] -= 1
checks_c[9] += 1
checks_c[8] += 1

check_sols = np.zeros((12,2))
for i, (check, check_c) in enumerate(zip(checks, checks_c)):
    check_sols[i][0] = np.round(check,4)
    check_sols[i][1] = np.round(check_c,4)

print(check_sols)
np.all(sols == check_sols)

[[0.775 0.3  ]
 [0.55  0.3  ]
 [0.825 0.35 ]
 [0.6   0.35 ]
 [0.75  0.5  ]
 [0.75  0.5  ]
 [0.575 0.35 ]
 [0.5   0.5  ]
 [0.775 0.55 ]
 [0.55  0.55 ]
 [0.575 0.35 ]
 [0.725 0.5  ]]


True

In [13]:
# creating gates
U = np.diag(U)
Uc = np.diag(Uc)

In [14]:
Ugate = qi.Operator(U).to_instruction()
Ugate.label = "CU"
CUgate = Ugate.control()

In [15]:
Ucgate = qi.Operator(Uc).to_instruction()
Ucgate.label = "CU'"
CUcgate = Ucgate.control()

In [16]:
# Single Cycle Representation
def bitstring_converter(string):
    values = []
    value = 0
    j = 0
    for i, v in enumerate(string):
        if v == '1':
            value += 1/(2**(i+1-j))
        elif v == " ":
            values.append(value)
            value = 0
            j = i+1
        if i == len(string)-1:
            values.append(value)
    return values[::-1]



def SingleHamiltonianCycle(eig, n):
    # we need a register for the eigenstate:
    eigst   = QuantumRegister(m, name = 'eigenstate')
    
    # we need two registers for the constrained problem:
    phase   = QuantumRegister(n, name = 'phase')
    phase_c = QuantumRegister(n, name = 'phase c')
    cr      = ClassicalRegister(n, 'output')
    cr_c    = ClassicalRegister(n, 'output c')
    
    # constructing the circuit (Initialization):
    qc = QuantumCircuit(phase, phase_c,eigst, cr,cr_c)
    
    # Apply H-Gates to phase qubits:
    for qubit in range(2*n):
        qc.h(qubit)
            
    for ind, val in enumerate(eig):
        if(int(val)):
            qc.x(ind + 2*n)
    
    ## Phase Estimation
    eig_qubits = np.arange(0,m) + 2*n
    
    repetitions = 1
    for counting_qubit in range(2*n):
        if counting_qubit == n:
            repetitions = 1
            qc.append(QFT(num_qubits = n, inverse = True, do_swaps=True), phase)
            qc.barrier()
        applied_qubits = np.append([counting_qubit], [eig_qubits])
        for i in range(repetitions):
            if counting_qubit < n:
                qc.append(CUgate, list(applied_qubits)); # This is CU
            else:
                qc.append(CUcgate, list(applied_qubits));
        repetitions *= 2
        
    #qc.append(QFT(num_qubits = n, inverse = True, do_swaps=True), phase)
    qc.append(QFT(num_qubits = n, inverse = True, do_swaps=True), phase_c)
    qc.barrier()
    
    qc.measure(phase,cr)
    qc.measure(phase_c,cr_c)

    return qc

In [17]:
n = 3 ## number of estimation qubits. 

eigstatelist = table[:,4]
for i,eig in enumerate(eigstatelist):
    while len(eig) < 12:
        eigstatelist[i] = '0' + eig
        eig = eigstatelist[i]

In [18]:
simulator = Aer.get_backend('qasm_simulator')
counts = []
for eig in eigstatelist:
    eig = eig[::-1] # needs to be reversed (Qiskit convention)
    qc = SingleHamiltonianCycle(eig, n)
    qc = transpile(qc, simulator)
    result = simulator.run(qc).result()
    counts.append(result.get_counts(qc))

In [36]:
eigstatelist

array(['001111001010', '001111010010', '010001010110', '001111111110',
       '010010110010', '010001110010', '011010000110', '011011110010',
       '011100100110', '011010111110', '100010110110', '100101101010'],
      dtype='<U21')

In [22]:
#df = pd.DataFrame

In [24]:
for i, count in enumerate(counts):
    if i == 0:
        df = pd.DataFrame(list(count.items()), columns=[str(i+1)+'-states', str(i+1)+'-counts'])
    else:
        dfc = pd.DataFrame(list(count.items()), columns=[str(i+1)+'-states', str(i+1)+'-counts'])
        df = pd.concat([df, dfc], axis=1)

In [28]:
df.head()

,1-states,1-counts,2-states,2-counts,3-states,3-counts,4-states,4-counts,5-states,5-counts,...,8-states,8-counts,9-states,9-counts,10-states,10-counts,11-states,11-counts,12-states,12-counts
0,101 000,1.0,001 000,1,001 011,1.0,010 101,51.0,100 110,1024.0,...,100 100,1024.0,101 010,1.0,110 000,1.0,111 110,1.0,100 100,14.0
1,100 111,1.0,011 110,8,010 010,2.0,000 101,7.0,NaN,NaN,...,NaN,NaN,110 010,1.0,110 110,1.0,101 010,1.0,100 001,6.0
2,001 011,1.0,011 000,3,100 101,3.0,010 100,1.0,NaN,NaN,...,NaN,NaN,101 011,1.0,010 011,2.0,101 000,1.0,100 010,6.0
3,100 010,1.0,101 110,1,101 101,1.0,100 110,2.0,NaN,NaN,...,NaN,NaN,111 111,1.0,000 100,4.0,010 010,2.0,100 111,24.0
4,011 100,1.0,110 100,9,011 011,11.0,010 010,2.0,NaN,NaN,...,NaN,NaN,110 001,1.0,000 010,1.0,110 100,2.0,100 110,906.0


In [29]:
filename = 'data/' + str(n) + 'qubit-5city.csv'
df.to_csv(filename, index=False)

In [ ]:
ns = [4,5,3]

for n in ns:
    counts = []
    for eig in eigstatelist:
        eig = eig[::-1] # needs to be reversed (Qiskit convention)
        qc = SingleHamiltonianCycle(eig, n)
        qc = transpile(qc, simulator)
        result = simulator.run(qc).result()
        counts.append(result.get_counts(qc))